# 6.2. Случайный лес

Ваша задача - написать класс `random_forest` для решения задачи классификации на основе датасета Ирисов Фишера (`sklearn.datasets.load_iris`), принимающий на вход конструктора аргументы `n_estimators`, `max_depth`, `subspaces_dim` и `random_state`. описание этих аргументов приведено ниже. У этого класса должны быть определены методы `.fit()` и `.predict()`, а также поле `._estimators`, в котором должен храниться список алгоритмов, используемых в ансамбле. 

Для создания обучающей подвыборки для каждого из базовых классификаторов, Вы можете воспользоваться классом `sample`, который Вы реализовали в прошлом задании. В случае его использования, не забудьте включить его описание в файл с Вашим решением текущего задания. Мы также предлагаем вам организовать выбор подпространств для каждого дерева посредством заполнения списка `subspace_idx`, в котором будут логироваться выбранные для каждого базового классификатора подпространства.

Замечание: в рамках выполнения данного задания запрещено использовать класс `sklearn.ensemble.RandomForestClassifier`. Такой код не пройдёт проверку.

Подберите также гиперпараметры, на которых ваш алгоритм получает наилучшее качество (с точки зрения метрики accuracy, доли правильных ответов) на тестовой выборке с параметром `test_size`=0.3, задайте их в виде глобальных переменных N_ESTIMATORS, MAX_DEPTH, SUBSPACE_DIM.

Шаблон класса:

In [1]:

import numpy as np
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from collections import Counter


class sample:
    def __init__(self, X: np.array, n_subspace: int):
        self.idx_subspace = self.random_subspace(X, n_subspace)

    def __call__(self, X: np.array, y: np.array):
        idx_obj = self.bootstrap_sample(X)
        X_sampled, y_sampled = self.get_subsample(
            X, y, self.idx_subspace, idx_obj
        )
        return X_sampled, y_sampled

    @staticmethod
    def bootstrap_sample(X: np.array):
        return np.unique(np.random.choice(X.shape[0], X.shape[0]))

    @staticmethod
    def random_subspace(X: np.array, n_subspace: int):
        return np.sort(np.random.choice(X.shape[1], n_subspace, replace=False))

    @staticmethod
    def get_subsample(
        X: np.array, y: np.array, idx_subspace: np.array, idx_obj: np.array
    ):
        return X[idx_obj][:, idx_subspace], y[idx_obj]
class random_forest:
    def __init__(
        self,
        n_estimators: int,
        max_depth: int,
        subspaces_dim: int,
        random_state: int,
    ):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.subspaces_dim = subspaces_dim
        self.random_state = random_state
        self.Classifier = []
        self.subspace_idx = []

    def fit(self, X: np.array, y: np.array):
        for _ in range(self.n_estimators):
            s = sample(X, self.subspaces_dim)
            X_sample, y_sample = s(X, y)
            self.Classifier.append(
                DecisionTreeClassifier(max_depth=self.max_depth).fit(
                    X_sample, y_sample
                )
            )
            self.subspace_idx.append(s.idx_subspace)

    def predict(self, X: np.array):
        tmp = np.empty((self.n_estimators, X.shape[0]), dtype=np.float64)

        for i in range(self.n_estimators):
            tmp[i, :] = self.Classifier[i].predict(X[:, self.subspace_idx[i]])

        out = tmp.T
        predicted = np.zeros(X.shape[0], dtype=int)

        for i, x in enumerate(out):
            counter = Counter(x)
            predicted[i] = counter.most_common(1)[0][0]

        return predicted
X, y = load_iris(return_X_y=True)
X_train, x_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, shuffle=True, random_state=42
)
accuracy = np.empty((20, 20, 4), dtype=np.float64)

for n_estimators in range(1, 21):
    for max_depth in range(1, 21):
        for subspace_dim in range(1, 5):
            clf = random_forest(n_estimators, max_depth, subspace_dim, 42)
            clf.fit(X_train, y_train)
            y_pred = clf.predict(x_test)
            accuracy[
                n_estimators - 1, max_depth - 1, subspace_dim - 1
            ] = accuracy_score(y_test, y_pred)
 
ind = np.unravel_index(np.argmax(accuracy, axis=None), accuracy.shape)
print(ind)
print(accuracy[ind[0], ind[1], ind[2]])
(0, 1, 2)
1.0
 

(0, 1, 1)
1.0


1.0

## Примечания

1. В данной задаче запрещено использовать библиотеку pandas.

2. В реализуемых методах запрещается использовать вывод любой информации на экран (в частности, недопустимо использование print()).

# Пример входных и выходных данных

In [2]:
X

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2],
       [5.4, 3.9, 1.7, 0.4],
       [4.6, 3.4, 1.4, 0.3],
       [5. , 3.4, 1.5, 0.2],
       [4.4, 2.9, 1.4, 0.2],
       [4.9, 3.1, 1.5, 0.1],
       [5.4, 3.7, 1.5, 0.2],
       [4.8, 3.4, 1.6, 0.2],
       [4.8, 3. , 1.4, 0.1],
       [4.3, 3. , 1.1, 0.1],
       [5.8, 4. , 1.2, 0.2],
       [5.7, 4.4, 1.5, 0.4],
       [5.4, 3.9, 1.3, 0.4],
       [5.1, 3.5, 1.4, 0.3],
       [5.7, 3.8, 1.7, 0.3],
       [5.1, 3.8, 1.5, 0.3],
       [5.4, 3.4, 1.7, 0.2],
       [5.1, 3.7, 1.5, 0.4],
       [4.6, 3.6, 1. , 0.2],
       [5.1, 3.3, 1.7, 0.5],
       [4.8, 3.4, 1.9, 0.2],
       [5. , 3. , 1.6, 0.2],
       [5. , 3.4, 1.6, 0.4],
       [5.2, 3.5, 1.5, 0.2],
       [5.2, 3.4, 1.4, 0.2],
       [4.7, 3.2, 1.6, 0.2],
       [4.8, 3.1, 1.6, 0.2],
       [5.4, 3.4, 1.5, 0.4],
       [5.2, 4.1, 1.5, 0.1],
       [5.5, 4.2, 1.4, 0.2],
       [4.9, 3

In [3]:
X.shape

(150, 4)

In [4]:
y.shape

(150,)

In [5]:
rf = random_forest(25, 15, 2, 42)
rf.fit(X, y)

In [6]:
preds = rf.predict(X)

In [7]:
preds

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])